<a href="https://colab.research.google.com/github/AntonioFuziy/DNA_mutation_detector/blob/master/RelatorioFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relatório Final - Supercomputação

### Paralelismo em GPU e CPU

Aluno: Antonio Fuziy

Prof: Luciano Silva

___

## Introdução

Para realização desse relatório, é necessário realizar uma contextualização sobre a implementação da busca exaustiva. Para a implementação da busca exaustiva a ideia é muito focada em exploration, ou seja ela não utiliza de otimizações para chegar a um resultado, dessa forma ele gera todas as possíveis subsequências a fim de conseguir encontrar a melhor pontuação a partir das duas sequências de entrada.

Para esse projeto, a ideia era utilizar a implementação sequencial da busca exaustiva para compara-la com os algoritmos de busca exaustiva paralelizados tanto em CPU quanto em GPU.

O paralelismo da CPU foi realizado utilizando uma biblioteca de `C++` chamada OpenMP, ao passo em que o paralelismo realizado utilizando a GPU foi utilizado a biblioteca **CUDA**, uma forma de construit algoritmos em paralelo programando em GPU. Portanto teriamos ao fim 3 implementações dentre elas, a busca exaustiva sequencial, a busca exaustiva paralelizada em CPU e a busca exaustiva paralelizada e GPU.

Segue a implementação das três abaixo:

**OBS: Foi necessário utilizar os códigos das implementações no notebook do Google Colab para gerar os executáveis para rodar no notebook pelo browser.**

In [ ]:
%%writefile exaustive_search.cpp
#include <iostream>
#include <vector>
#include <string>
#include <algorithm>
#include <random>
#include <fstream>

using namespace std;

struct comp_seqs {
  string a;
  string b;
  int score;
};

// void reportTime(const char* msg, steady_clock::duration span) {
//   auto ms = duration_cast<milliseconds>(span);
//   cout << msg << " - levou - " <<
//   ms.count() << " milisegundos" << endl;
// }

int match(char a, char b){
  if(a == b){
    return 2;
  }
  return -1;
}

// int calculate_score(string a, string b){
//   int score = 0;
//   for(int i = 0; i < int(a.size()); i++){
//     score += match(a[i], b[i]);
//   }
//   return score;
// }
int return_index(int a, int b, int c){
  if(a >= b and a >= c and a >= 0){
    return 1;
  } else if(b >= c and b >= 0){
    return 2;
  } else if(c >= 0){
    return 3;
  }
  return 0;
}

vector<string> generate_subsequences(string sequence){
  vector<string> subsequences;

  for(int i = 0; i <= int(sequence.size()); i++){
    for(int j = 1; j <= int(sequence.size()); j++){
      subsequences.push_back(sequence.substr(i, j));
    }
  }

  return subsequences;
}

int calculate_score(string a, string b, int n, int m, vector<vector<int>> H){
  int max_H = 0;
  int w;
  int diagonal, delecao, insercao, current_index;
  
  for (int i = 1; i <= n; i++){
    for (int j = 1; j <= m; j++){
      w = match(a[i-1], b[j-1]);
      diagonal = H[i-1][j-1] + w;
      delecao = H[i-1][j] - 1;
      insercao = H[i][j-1] - 1;
      current_index = return_index(diagonal, delecao, insercao);

      if(current_index == 1){
        H[i][j] = diagonal;
      } else if(current_index == 2){
        H[i][j] = delecao;
      } else if(current_index == 3){
        H[i][j] = insercao;
      } 
      else {
        H[i][j] = 0;
      }

      if(H[i][j] > max_H){
        max_H = H[i][j];
      }
    }
  }
  return max_H;
}

int main(){
  int n;
  int m;
  string a; 
  string b;
  int score = 0;
  vector<vector<int>> H;

  cin >> n >> m;
  cin >> a >> b;

  cout << "A size: " << n << endl;
  cout << "B size: " << m << endl;
  cout << "" << endl;
  cout << "A: " << a << endl;
  cout << "B: " << b << endl;
  cout << "" << endl;

  H.resize(n+1);
  for(int i = 0; i < n+1; i++){
    H[i].resize(m+1);
  }
  
  vector<string> subsequences_a = generate_subsequences(a);
  vector<string> subsequences_b = generate_subsequences(b);

  string subsequence_a;
  string subsequence_b;

  string greater_sequence;
  string minor_sequence;

  vector<comp_seqs> sequences_result;

  for(int i = 0; i < int(subsequences_a.size()); i++){
    for(int j = 0; j < int(subsequences_b.size()); j++){
      if(int(subsequences_a[i].size()) == int(subsequences_b[j].size())){
        subsequence_a = subsequences_a[i];
        subsequence_b = subsequences_b[j];

        comp_seqs sequence_result;
        sequence_result.a = subsequence_a;
        sequence_result.b = subsequence_b;
        sequences_result.push_back(sequence_result);
      }
    }
  }

  comp_seqs best_sequences;
  best_sequences.score = 0;
  for (int i = 0; i < int(sequences_result.size()); i++){
    score = calculate_score(sequences_result[i].a, sequences_result[i].b, n, m, H);
    sequences_result[i].score = score;
    if(sequences_result[i].score > best_sequences.score){
      best_sequences = sequences_result[i];
    }
  }

  cout << "Score: " << best_sequences.score << endl;

  return 0;
}

Overwriting exaustive_search.cpp


In [ ]:
%%writefile openmp_exaustive_search.cpp
#include <iostream>
#include <vector>
#include <string>
#include <algorithm>
#include <random>
#include <fstream>

using namespace std;

struct comp_seqs {
  string a;
  string b;
  int score;
};

// void reportTime(const char* msg, steady_clock::duration span) {
//   auto ms = duration_cast<milliseconds>(span);
//   cout << msg << " - levou - " <<
//   ms.count() << " milisegundos" << endl;
// }

int match(char a, char b){
  if(a == b){
    return 2;
  }
  return -1;
}

// int calculate_score(string a, string b){
//   int score = 0;
//   for(int i = 0; i < int(a.size()); i++){
//     score += match(a[i], b[i]);
//   }
//   return score;
// }
int return_index(int a, int b, int c){
  if(a >= b and a >= c and a >= 0){
    return 1;
  } else if(b >= c and b >= 0){
    return 2;
  } else if(c >= 0){
    return 3;
  }
  return 0;
}

vector<string> generate_subsequences(string sequence){
  vector<string> subsequences;

  for(int i = 0; i <= int(sequence.size()); i++){
    for(int j = 1; j <= int(sequence.size()); j++){
      subsequences.push_back(sequence.substr(i, j));
    }
  }

  return subsequences;
}

int calculate_score(string a, string b, int n, int m, vector<vector<int>> H){
  int max_H = 0;
  int w;
  int diagonal, delecao, insercao, current_index;
  
  for (int i = 1; i <= n; i++){
    for (int j = 1; j <= m; j++){
      w = match(a[i-1], b[j-1]);
      diagonal = H[i-1][j-1] + w;
      delecao = H[i-1][j] - 1;
      insercao = H[i][j-1] - 1;
      current_index = return_index(diagonal, delecao, insercao);

      if(current_index == 1){
        H[i][j] = diagonal;
      } else if(current_index == 2){
        H[i][j] = delecao;
      } else if(current_index == 3){
        H[i][j] = insercao;
      } 
      else {
        H[i][j] = 0;
      }

      if(H[i][j] > max_H){
        max_H = H[i][j];
      }
    }
  }
  return max_H;
}

int main(){
  int n;
  int m;
  string a; 
  string b;
  int score = 0;
  vector<vector<int>> H;

  cin >> n >> m;
  cin >> a >> b;

  cout << "A size: " << n << endl;
  cout << "B size: " << m << endl;
  cout << "" << endl;
  cout << "A: " << a << endl;
  cout << "B: " << b << endl;
  cout << "" << endl;

  H.resize(n+1);
  for(int i = 0; i < n+1; i++){
    H[i].resize(m+1);
  }
  
  vector<string> subsequences_a = generate_subsequences(a);
  vector<string> subsequences_b = generate_subsequences(b);

  string subsequence_a;
  string subsequence_b;

  string greater_sequence;
  string minor_sequence;

  vector<comp_seqs> sequences_result;

  for(int i = 0; i < int(subsequences_a.size()); i++){
    for(int j = 0; j < int(subsequences_b.size()); j++){
      if(int(subsequences_a[i].size()) == int(subsequences_b[j].size())){
        subsequence_a = subsequences_a[i];
        subsequence_b = subsequences_b[j];

        comp_seqs sequence_result;
        sequence_result.a = subsequence_a;
        sequence_result.b = subsequence_b;
        sequences_result.push_back(sequence_result);
      }
    }
  }

  comp_seqs best_sequences;
  best_sequences.score = 0;
  for (int i = 0; i < int(sequences_result.size()); i++){
    score = calculate_score(sequences_result[i].a, sequences_result[i].b, n, m, H);
    sequences_result[i].score = score;
    if(sequences_result[i].score > best_sequences.score){
      best_sequences = sequences_result[i];
    }
  }

  cout << "Score: " << best_sequences.score << endl;

  return 0;
}

Overwriting openmp_exaustive_search.cpp


In [ ]:
%%writefile gpu_exaustive_search.cu
#include <iostream>
#include <algorithm>
#include <iomanip>
#include <cstdlib>
#include <chrono>
#include <cstdlib>
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/functional.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <thrust/fill.h>

using namespace std;

struct begin_end{
  int start_a;
  int start_b;
  int size_a;
  int size_b;
};

using namespace std::chrono;

std::vector<begin_end> generate_indexes(int max_size, int min_size){
  vector<begin_end> indexes;
  begin_end current_index;

  for(int index = 0; index < min_size; index++){
    for(int i = 0; i < max_size; i++){
      for(int j = 0; j < min_size; j++){
        current_index.start_a = i;
        current_index.start_b = j;
        current_index.size_a = i+index;
        current_index.size_b = j+index;
        indexes.push_back(current_index);
      }
    }
  }
  return indexes;
}

void reportTime(const char* msg, steady_clock::duration span) {
  auto ms = duration_cast<milliseconds>(span);
  std::cout << msg << " - levou - " <<
  ms.count() << " milisegundos" << std::endl;
}

struct sequence_match{
  __host__ __device__
  int operator()(const char& a, const char& b){
    if(a == b){
      return 2;
    }
    return -1;
  }
};

int main(){
  int m, n;
  string a, b;
  steady_clock::time_point ts, te;
  
  std::cin >> n;
  std::cin >> m;
  std::cin >> a;
  std::cin >> b;

  int max_size = 0;
  int min_size = 0;
  int max_score = 0;

  min_size = int(a.size());
  max_size = int(b.size());

  std::vector<begin_end> all_indexes = generate_indexes(max_size, min_size);

  thrust::device_vector<char> a_gpu(n);
  thrust::device_vector<char> b_gpu(m);
  thrust::device_vector<char> all_sequences(min_size);

  for(int i = 0; i < n; i++){
    a_gpu[i] = a[i];
  }

  for(int i = 0; i < m; i++){
    b_gpu[i] = b[i];
  }

  ts = steady_clock::now();
  for(int i = 0; i < int(all_indexes.size()); i++){
    thrust::transform(
      a_gpu.begin() + all_indexes[i].start_a, a_gpu.end() + all_indexes[i].size_a,
      b_gpu.begin() + all_indexes[i].start_b, all_sequences.begin(),
      sequence_match()
    );
   
    int score = thrust::reduce(all_sequences.begin(), all_sequences.end(), (int) 0, thrust::plus<int>());
    if (max_score < score){
      max_score = score;
    }
  }
  
  te = steady_clock::now();
  reportTime("Tempo para calculo", te - ts);
  std::cout << std::fixed << std::setprecision(4);

  std::cout << "Score: " << max_score << endl;

  return 0;
}

Overwriting gpu_exaustive_search.cu


Compilando os códigos para gerar os executáveis

In [ ]:
!g++ -Wall -O3 exaustive_search.cpp -o exaustive_search
!g++ -g -Wall -fopenmp openmp_exaustive_search.cpp -o openmp_exaustive_search
!nvcc -arch=sm_37 -std=c++14 gpu_exaustive_search.cu -o gpu_exaustive_search

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


#### Algumas orientações para rodar os códigos

Para gerar as subsequências de teste utilizou-se um script em python chamado `sequence_generator.py` o qual gera várias subsequências para serem testadas pelos 3 algoritmos citados anteriormente. Para que o teste das sequências seja possível, é necessário criar um diretório `dna_sequences`, adicionar nesse diretório o script `sequence_generator.py` e dentro desse mesmo diretório criar uma pasta `all_sequences`, assim o script em python consegue gerar multiplas subsequências como o notebook necessita.  

In [ ]:
#executando o script que gera as subsequencias
!python3 ./dna_sequences/sequence_generator.py

#### Execução do teste dos algoritmos

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import subprocess
plt.style.use("ggplot")

In [ ]:
algorithms = [
    "/content/exaustive_search",
    "/content/openmp_exaustive_search",
    "/content/gpu_exaustive_search"
]

input_directory = "dna_sequences/all_sequences"

In [ ]:
def roda_com_entrada(executavel, algorithm):
  with open(algorithm) as f:
    start = time.perf_counter()
    proc = subprocess.run([executavel], input=f.read(), text=True, capture_output=True)
    end = time.perf_counter()
  return end-start

In [ ]:
def generate_time(algorithm, directory):
    tempos_busca = []
    dir_list = os.listdir(directory)
    for f in dir_list:
        tempos_busca.append(roda_com_entrada(algorithm,f'{directory}/{f}'))
    return tempos_busca, dir_list

In [ ]:
def generate_length(directory, dir_list):
    n = []
    m = []
    for file in dir_list:
        with open(f'{directory}/{file}') as f :
            text_splitted = f.read().splitlines()
        n.append(text_splitted[0])
        m.append(text_splitted[1])
    n = [int(i) for i in n]
    m = [int(j) for j in m]
    return n, m

Rodando os exeutáveis dos três algoritmos para testar as sequências

In [ ]:
tempos_exaustiva_sequencial, dir_exaustiva_sequencial = generate_time(algorithms[0], input_directory)
n_exaustiva_sequencial, m_exaustiva_sequencial = generate_length(input_directory, dir_exaustiva_sequencial)

tempos_exaustiva_parallel, dir_exaustiva_parallel  = generate_time(algorithms[1], input_directory)
n_exaustiva_parallel, m_exaustiva_parallel = generate_length(input_directory, dir_exaustiva_parallel)

tempos_exaustiva_gpu, dir_exaustiva_gpu  = generate_time(algorithms[2], input_directory)
n_exaustiva_gpu, m_exaustiva_gpu = generate_length(input_directory, dir_exaustiva_gpu)

**Gráfico que compara as três implementações da busca exaustiva, conforme as sequências de entrada.**

In [ ]:
plt.figure(figsize=(12, 7))
# plt.scatter(n_exaustiva_sequencial, tempos_exaustiva_sequencial, label="sequential")
# plt.scatter(n_exaustiva_parallel, tempos_exaustiva_parallel, label="openmp parallel")
plt.scatter(n_exaustiva_gpu, tempos_exaustiva_gpu, label="gpu parallel")
plt.title("Exaustive Search")
plt.xlabel('Sequência')
plt.ylabel("Tempo")
plt.legend()

plt.show()

![GPU](https://github.com/AntonioFuziy/DNA_mutation_detector/blob/master/parallel/gpu/gpu.png?raw=true)

## Análise dos resultados

A partir dos resultados obtidos, pode-se tirar algumas conclusões sobre os algoritmos implementados.

Primeiramente a partir do gráfico acima comparando-se o algoritmo sequencial da busca exaustiva com o paralelo em OpenMP, é possível verificar que o desempenho para sequências pequenas com tamanho até mais ou menos 77, é muito parecido sem tanto ganho de performance, no momento em que os algoritmos testam as sequências maiores, o ganho de performance se mostra mais presente, porém não tão significante, mostrando que o algoritmo utilizando OpenMP tem um ganho maior em relação ao tempo do algoritmo sequencial.

Enquanto isso, observando o algoritmo que utiliza de GPU, pode-se perceber que sua performance não se apresenta tão significativa para sequências de tamanho menor, mas quando o tamanho das sequências aumentam de forma significativa, o tempo de execução do algortimo utilizando a GPU torna-se muito mais rápido que o algoritmo sequencial e o paralelo em OpenMP, é possível ver isso pelas curvas do gráfico acima, de forma que o paralelismo em GPU ganha exponecialmente quando as sequências ficam cada vez maiores.

Por fim, quando olhamos para sequêcias de tamanho pequeno nenhum dos três algoritmos apresenta uma performance muito vantajosa quando comparados um ao outro, porém quando os tamanhos das sequências aumentam, o tempo de execução do paralelismo em GPU começa a ganhar exponencialmente dos outros dois algoritmos, mesmo que o paralelismo em OpenMP consiga ganhar do algoritmo sequencial. Dessa forma, pode-se concluir que para sequências pequenas ambos os algoritmos podem ser utilizados sem tanta perda de performance, porém para sequências muito grandes o algoritmo paralelizado em GPU torna-se altamente recomendável, uma vez que sua performance ganha exponencialmente dos demais algoritmos.